## Evaluating the Fabric Data Agent

### 1. Initial Configurations
#### 1.1 Libraries and packages

In [1]:
# Install initial packages and sdk
%pip install openai==1.70.0 httpx==0.27.2

StatementMeta(, 8e753cac-58dd-44d0-bd97-b418b6932841, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.0/349.0 kB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.9/444.9 kB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 146.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Not uninstalling typing-extensions at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-6428c09e-7214-4dfe-8e16-51453d64fe16
    Can't uninstall 'typing_extensions'. No files were found to uninsta

In [2]:
%pip install -U fabric-data-agent-sdk

StatementMeta(, 8e753cac-58dd-44d0-bd97-b418b6932841, 14, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 6.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of azure-kusto-data to determine which version is compatible with other requirements. This could take a while.
Reason for being yanked: Yanked due to conflicts with CVE-2024-35195 mitigation
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.3/699.3 kB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existi

In [7]:
# Install generic libraries and packages
import requests
import json
import pprint
import typing as t
import time
import uuid
import pandas as pd

# Install openai libraries and packages
import openai, httpx
from openai import OpenAI
from openai._exceptions import APIStatusError
from openai._models import FinalRequestOptions
from openai._types import Omit
from openai._utils import is_given
from synapse.ml.mlflow import get_mlflow_env_config
from sempy.fabric._token_provider import SynapseTokenProvider
from fabric.dataagent.evaluation import evaluate_data_agent #evaluacion del agente
from fabric.dataagent.evaluation import get_evaluation_summary #obtener el resumen de evaluacion

StatementMeta(, 8e753cac-58dd-44d0-bd97-b418b6932841, 20, Finished, Available, Finished)

### 2. Define the questions to ask to the Agent

Now, we are going to define some examples of questions (with their respective answers) that we are goingo to use during the evaluation

In [4]:
# Define a sample evaluation set with user questions and their expected answers.
df = pd.DataFrame(
    columns=["question", "expected_answer"],
    data=[
        ["Dime el resultado neto que se ha obtenido en diciembre de 2025", "141.492"],
        ["¿Cuál es el cliente con mayor volumen de ventas para la comunidad autónoma (CCAA) de Valencia en 2025?", "Carpintería Martínez"],
        ["Muestrame el valor acumulado del EBITD para julio de 2024", "3.237.742"],
        ["¿Cuál es el tercer producto con mayor volumen de ventas para la comunidad autónoma (CCAA) de Galicia en 2025?", "Serrucho manual"]
    ]
)

df.head()

StatementMeta(, 8e753cac-58dd-44d0-bd97-b418b6932841, 17, Finished, Available, Finished)

,question,expected_answer
0,Dime el resultado neto que se ha obtenido en d...,141.492
1,¿Cuál es el cliente con mayor volumen de venta...,Carpintería Martínez
2,Muestrame el valor acumulado del EBITD para ju...,3.237.742
3,¿Cuál es el tercer producto con mayor volumen ...,Serrucho manual


### 3. Evaluate the Agent

The next step is to run the evaluation using the evaluate_data_agent function. This function compares the agent's responses with the expected results and stores the evaluation metrics.

In [5]:
# Name of your Data Agent
data_agent_name = "FinancialAnalystAssistant_FDA"

# (Optional) Name of the workspace if the Data Agent is in a different workspace
#workspace_name = None

# (Optional) Name of the output table to store evaluation results (default: "evaluation_output")
# Two tables will be created:
# - "<table_name>": contains summary results (e.g., accuracy)
# - "<table_name>_steps": contains detailed reasoning and step-by-step execution
table_name = "demo_evaluation_output"

# Specify the Data Agent stage: "production" (default) or "sandbox"
data_agent_stage = "production"

# Run the evaluation and get the evaluation ID
evaluation_id = evaluate_data_agent(
    df,
    data_agent_name,
    #workspace_name=workspace_name,
    table_name=table_name,
    data_agent_stage=data_agent_stage
)

print(f"Unique ID for the current evaluation run: {evaluation_id}")

StatementMeta(, 8e753cac-58dd-44d0-bd97-b418b6932841, 18, Finished, Available, Finished)

**🔗Failed Thread(s):**
- [thread_4qs87rNoJkii1hoo60tk5CnB](https://app.fabric.microsoft.com/workloads/de-ds/dataagents/80caf30a-9a48-4a48-81fd-f3762bba1365/externalThread?debug.aiSkillThreadIdOverride=thread_4qs87rNoJkii1hoo60tk5CnB&debug.aiSkillViewPublishedOverride=0)
- [thread_N8A2aOGrYiLJYDEgQHdZFRAc](https://app.fabric.microsoft.com/workloads/de-ds/dataagents/80caf30a-9a48-4a48-81fd-f3762bba1365/externalThread?debug.aiSkillThreadIdOverride=thread_N8A2aOGrYiLJYDEgQHdZFRAc&debug.aiSkillViewPublishedOverride=0)
- [thread_rP1RsbQODMIn24VAvzASyl7V](https://app.fabric.microsoft.com/workloads/de-ds/dataagents/80caf30a-9a48-4a48-81fd-f3762bba1365/externalThread?debug.aiSkillThreadIdOverride=thread_rP1RsbQODMIn24VAvzASyl7V&debug.aiSkillViewPublishedOverride=0)
- [thread_Em5D1EX201clPJ13r06tXdkp](https://app.fabric.microsoft.com/workloads/de-ds/dataagents/80caf30a-9a48-4a48-81fd-f3762bba1365/externalThread?debug.aiSkillThreadIdOverride=thread_Em5D1EX201clPJ13r06tXdkp&debug.aiSkillViewPublishedOverride=0)

100%|██████████| 4/4 [00:30<00:00,  7.60s/it]


Unique ID for the current evaluation run: c5d79162-6e3f-4e57-9dcc-6819fc5a4c87


We can have a summary of the results evaluation, in which we can see if the responses of the Agent are true, false or doubtful

In [9]:
# Retrieve a summary of the evaluation results
df = get_evaluation_summary(table_name)
df.head()

StatementMeta(, 8e753cac-58dd-44d0-bd97-b418b6932841, 22, Finished, Available, Finished)

index,evaluation_id,T,F,?,%
0,c5d79162-6e3f-4e57-9dcc-6819fc5a4c87,0,4,0,0.0


### 4. Check the results deeper

To delve deeper into how the Data Agent responded to each individual question, use the get_evaluation_details function. This function returns a detailed breakdown of the evaluation run, including the agent's actual responses, whether they match the expected answer, and a link to the evaluation thread.

In [12]:
from fabric.dataagent.evaluation import get_evaluation_details

# Table name used during evaluation
table_name = "demo_evaluation_output"

# Whether to return all evaluation rows (True) or only failures (False)
get_all_rows = False

# Whether to print a summary of the results
verbose = True

# Retrieve evaluation details for a specific run
eval_details = get_evaluation_details(
    evaluation_id,
    table_name,
    get_all_rows=get_all_rows,
    verbose=verbose
)

StatementMeta(, 8e753cac-58dd-44d0-bd97-b418b6932841, 25, Finished, Available, Finished)

index,question,expected_answer,actual_answer,evaluation_judgement,thread_url
0,Dime el resultado neto que se ha obtenido en diciembre de 2025,141.492,"No dispongo del dato del Resultado Neto para diciembre de 2025 con la información disponible actualmente. Si necesitas el resultado de otro mes o año, házmelo saber.",False,thread_4qs87rNoJkii1hoo60tk5CnB
1,Muestrame el valor acumulado del EBITD para julio de 2024,3.237.742,El valor acumulado del EBITDA para julio de 2024 es de 1.526.084 €.,False,thread_rP1RsbQODMIn24VAvzASyl7V
2,¿Cuál es el cliente con mayor volumen de ventas para la comunidad autónoma (CCAA) de Valencia en 2025?,Carpintería Martínez,"No se han encontrado datos de ventas para la comunidad autónoma de Valencia en el año 2025, o bien aún no existen registros para ese periodo o región en el sistema. Si necesitas información de otro año o comunidad, por favor indícalo.",False,thread_Em5D1EX201clPJ13r06tXdkp
3,¿Cuál es el tercer producto con mayor volumen de ventas para la comunidad autónoma (CCAA) de Galicia en 2025?,Serrucho manual,El tercer producto con mayor volumen de ventas en Galicia en 2025 es el Poste tratado 8x8.,False,thread_N8A2aOGrYiLJYDEgQHdZFRAc
